# Results

In the below tables summarise the results for each dataset. 


The following represent the scenarios considered for the ML reconciliation approach:

```
CASE1 - When the validation loss considers only the bottom level time series

CASE2 - When the validation loss considers the entire hierarchy

```

For each CASE 1 and CASE 2 have different lambda configurations are carried out as below:

```
lambda_range = [0.01, 0.09]
lambda_range = [0.1, 0.9]
lambda_range = [1, 1.5]
lambda_range = [0.01, 5]
lambda_range = 1
```

Every data set has multiple samples

```
dataset_samples = {'prison': 3, 'tourism': 10, 'wikipedia': 10, 'labour': 5}
```

The results are also shown for full forecast horizon and half forecast horizon.
```
Prison : horizon = 8, half horizon = 4
Tourism: horizon = 12, half horizon = 6
Wikipedia: horizon = 7, half horizon = 3
Labour: horizon = 12, half horizon = 4
```

In [1]:
import pandas as pd

In [2]:
RESULTS_PATH = '../results/new_results/errors/'

In [3]:
def get_error_table(data_frame, LEVELS):
    errors = data_frame.groupby(by=['error_metric', 'level']).mean().reset_index().round(2)
    errors = pd.pivot(errors, values ='error', columns='error_metric', index='level')
    errors.index = LEVELS
    overall_error = data_frame.groupby(by='error_metric').mean()[['error']].transpose()
    overall_error.index = ['Overall']
    return errors.append(overall_error).round(2)

In [4]:
def get_percentage_improvement(base_df, approach_df):
    return (base_df - approach_df)/base_df*100

In [5]:
def get_results(data, model, LEVELS, FC_TYPES, short_horizon = False):
    error_dic = {}
    error_tables = {}
    error_percentages = {}
    final_percentage_tables = {}
    
    for fc_type in FC_TYPE:
        file_name = f'{data}_{model}'
        if short_horizon:
            file_name = f'{file_name}_short_horizon'
        error_dic[fc_type] = pd.read_csv(f'{RESULTS_PATH}{file_name}_{fc_type}.csv', index_col=0)
        
    for fc_name, error_df in error_dic.items():
        error_tables[fc_name] = get_error_table(error_df, LEVELS)
        
    for fc_name, error_df in error_tables.items():
        error_percentages[fc_name] = get_percentage_improvement(error_tables['base'], error_df)
        
    for error_metric in ['MSE', 'MAE', 'SMAPE']:
        dataframe_error = []
        for fc_name, error_df in error_percentages.items():
            if fc_name =='base':
                continue
            error_metric_percentages = error_df[[error_metric]]
            error_metric_percentages.columns = [fc_name]
            dataframe_error.append(error_metric_percentages)
        final_percentage_tables[error_metric] = pd.concat(dataframe_error, axis=1)[FC_TYPES[1:]]
        
    return final_percentage_tables

In [37]:
def get_results_across_samples(data, model, LEVELS, FC_TYPES, no_samples, short_horizon = False):
    sample_results = []
    for sample in range(0, no_samples):
        data_name = f'{data}_{sample}'
        per_sample_result = get_results(data_name, model, LEVELS, FC_TYPES, short_horizon)
        sample_results.append(per_sample_result['MSE'])
    sample_per = pd.concat(sample_results)

    return sample_per.groupby(sample_per.index).mean().reindex(sample_results[0].index.values)

## New Results

## Prison

In [20]:
LEVELS_prison = ['Australia', 'State', 'Gender', 'Legal', 'Indigenous']
data_prison = 'prison'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm',
               'case1_lambda_1_median', 'case1_lambda_1_mean',
               'case1_lambda_[0.01, 0.09]_mean', 'case1_lambda_[0.01, 0.09]_median',
               'case1_lambda_[0.1, 0.9]_mean', 'case1_lambda_[0.1, 0.9]_median',
               'case1_lambda_[1, 1.5]_mean', 'case1_lambda_[1, 1.5]_median',
               'case1_lambda_[0.01, 5]_mean', 'case1_lambda_[0.01, 5]_median',
               'case2_lambda_1_median', 'case2_lambda_1_mean',
               'case2_lambda_[0.01, 0.09]_mean', 'case2_lambda_[0.01, 0.09]_median',
               'case2_lambda_[0.1, 0.9]_mean', 'case2_lambda_[0.1, 0.9]_median',
               'case2_lambda_[1, 1.5]_mean', 'case2_lambda_[1, 1.5]_median',
               'case2_lambda_[0.01, 5]_mean', 'case2_lambda_[0.01, 5]_median']

model_arima ='arima'
percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE)



ARIMA

In [23]:
percentages_prison_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,mintshrink,ols,wls,erm,"case2_lambda_[1, 1.5]_mean","case2_lambda_[0.1, 0.9]_median","case2_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_median","case2_lambda_[0.1, 0.9]_mean","case1_lambda_[1, 1.5]_mean",...,case2_lambda_1_mean,"case2_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_median","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median",bottomup,"case1_lambda_[0.01, 0.09]_mean",case1_lambda_1_mean,case1_lambda_1_median,"case1_lambda_[0.01, 0.09]_median"
Australia,-114.450766,-13.275846,-230.437789,-150.126572,-678.348018,-809.328768,-700.305952,-776.492563,-804.938492,-818.617855,...,-859.830031,-886.962722,-924.186620,-1075.840201,-1100.031075,-1204.936229,-1173.292097,-1154.926767,-1187.037023,-1264.680111
State,20.707253,8.906267,14.764202,9.486221,7.552543,13.789161,6.817897,11.512080,12.288071,10.673523,...,3.653668,3.261425,1.716966,-5.545980,-5.178477,-0.741349,-3.506965,-9.556132,-10.476289,-8.675139
Gender,19.009192,10.048874,13.384368,15.612176,4.596505,12.528182,4.178300,8.360630,10.320190,8.473054,...,-0.068366,0.027648,-0.361493,-8.331175,-8.381930,-4.889860,-6.862056,-11.132581,-12.069117,-12.214011
Legal,-3.632026,-10.070241,-11.037498,-59.544452,-6.076829,0.325273,-6.110491,-3.219295,-2.171581,-3.573201,...,-9.819959,-9.679348,-9.023400,-15.127927,-15.207178,-12.232326,-12.632673,-16.119184,-18.380198,-16.125335
Indigenous,8.681385,1.381039,1.571899,-36.429845,4.010178,8.232931,4.303609,6.290674,6.447927,5.711140,...,1.530647,1.922401,2.102749,-1.860179,-1.833581,0.000000,-0.028934,-2.451234,-3.928738,-2.167201
Overall,7.575853,3.695151,-4.110239,-15.393930,-29.891465,-30.181053,-31.312970,-31.433872,-31.805042,-33.916015,...,-42.501341,-43.868192,-46.213524,-60.098661,-61.180577,-63.001288,-63.053055,-66.448091,-69.222360,-71.786323


ETS

In [25]:
model_ets ='ets'
percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE)
percentages_prison_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,case2_lambda_1_median,case2_lambda_1_mean,mintshrink,case1_lambda_1_median,case1_lambda_1_mean,wls,"case2_lambda_[0.01, 5]_mean","case1_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_median",...,"case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_mean","case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_median",bottomup,"case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean",erm
Australia,-95.390013,-1752.959398,-1674.247607,-932.717901,-1938.802433,-2004.772327,-1439.641421,-2478.836787,-2508.483361,-2508.483361,...,-2692.603123,-2655.540465,-3247.314494,-3247.314494,-3330.050909,-3330.050909,-3671.615741,-3412.641714,-3426.728418,-5300.929077
State,14.275249,37.465443,34.735666,20.751485,31.127975,27.492677,20.715633,31.332140,31.411291,31.411291,...,30.690589,28.138822,22.035281,22.035281,20.115481,20.115481,23.569975,5.528748,5.764668,-14.648431
Gender,3.129698,26.704002,24.208300,7.916996,20.738985,16.988272,7.906455,20.701586,20.134731,20.134731,...,20.151422,17.523523,11.959087,11.959087,9.234194,9.234194,12.276370,-5.691380,-5.873214,-34.866193
Legal,-0.466017,12.224498,10.438459,0.125546,8.318751,5.370089,-4.214015,9.775611,9.687670,9.687670,...,9.090782,6.141381,4.819727,4.819727,3.216468,3.216468,6.081990,-4.639609,-4.755457,-24.828064
Indigenous,-5.287354,5.034138,3.556016,-7.243976,1.895672,-0.740630,-14.794398,2.980222,2.952540,2.952540,...,2.386161,-0.262632,-1.003702,-1.003702,-2.291314,-2.291314,0.000000,-8.370457,-8.650539,-86.333464
Overall,4.501743,1.370642,0.146073,-3.108093,-6.441097,-10.699482,-11.767894,-13.287190,-13.833525,-13.833525,...,-16.800279,-18.924292,-31.251201,-31.251201,-34.349870,-34.349870,-35.948105,-47.610250,-47.813873,-102.595354


### Across 3 Samples

ARIMA

In [39]:
sample_precentages = get_results_across_samples(data_prison, model_arima, LEVELS_prison, FC_TYPE, 3)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,erm,wls,mintshrink,"case1_lambda_[0.1, 0.9]_mean","case2_lambda_[0.01, 0.09]_mean",case1_lambda_1_mean,"case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_mean","case1_lambda_[0.1, 0.9]_median",...,"case1_lambda_[1, 1.5]_mean",case2_lambda_1_mean,case2_lambda_1_median,"case1_lambda_[0.01, 5]_mean",bottomup,"case2_lambda_[0.1, 0.9]_median","case1_lambda_[0.01, 5]_median","case2_lambda_[0.01, 5]_median","case2_lambda_[0.1, 0.9]_mean","case2_lambda_[0.01, 5]_mean"
Australia,16.037273,13.945501,-4.427708,-19.960524,-10.036179,-11.910585,-15.223576,-13.062421,-15.464915,-14.883102,...,-44.882929,-35.386369,-31.694465,-52.955931,-34.189824,-34.976507,-62.339963,-56.679453,-42.877565,-60.458296
State,2.079091,-13.230265,-10.914427,-18.445520,-25.831448,-26.945447,-27.864748,-27.134962,-27.279844,-28.316349,...,-26.594839,-28.088734,-28.508172,-24.400378,-29.371348,-29.618081,-24.192767,-30.137315,-29.443158,-29.698380
Gender,5.544659,11.202332,3.701267,-2.633521,-1.582994,-0.955991,-1.524372,-1.323453,-1.786643,-3.271379,...,-2.333926,-3.922029,-4.508750,-2.170740,-5.281623,-5.483280,-2.413545,-3.886342,-5.715199,-3.954673
Legal,7.285606,7.614872,6.036748,3.505205,0.002399,-0.335983,-0.339994,-0.474429,-0.709203,-0.991621,...,-0.309250,-1.046983,-1.616105,-0.410700,-2.283569,-2.096699,-0.346148,-1.486043,-1.936269,-1.423011
Indigenous,6.773241,-0.078755,6.743089,6.766005,1.642781,1.516429,1.508648,1.413483,1.230420,0.773393,...,1.607481,0.995548,0.494169,1.308518,0.000000,0.153329,1.309625,0.628268,0.214601,0.713261
Overall,8.274700,1.872688,-0.734054,-6.794685,-11.456338,-12.399676,-12.660036,-12.691593,-13.284605,-13.328686,...,-15.668757,-15.869416,-16.065520,-17.342690,-17.542824,-17.690223,-18.381654,-18.498049,-18.659623,-18.844661


ETS

In [40]:
sample_precentages = get_results_across_samples(data_prison, model_ets, LEVELS_prison, FC_TYPE, 3)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,erm,mintshrink,case1_lambda_1_mean,"case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_mean",wls,"case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.1, 0.9]_mean",...,"case2_lambda_[0.01, 5]_mean","case2_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean","case1_lambda_[0.1, 0.9]_median","case2_lambda_[1, 1.5]_median","case1_lambda_[0.1, 0.9]_mean",case2_lambda_1_mean,case2_lambda_1_median,bottomup
Australia,-13.865741,-153.288971,-111.370860,-625.806268,-665.296933,-665.760578,-373.970171,-684.114237,-684.279937,-595.761386,...,-552.229119,-677.431651,-710.310311,-720.746352,-607.916165,-755.078001,-606.936039,-760.691271,-819.465447,-1020.031795
State,15.940048,12.794166,15.642751,17.459051,16.909560,16.831927,13.167508,13.586555,13.269311,12.124092,...,12.139927,12.055541,11.576171,10.659922,9.856314,10.851984,9.586130,9.335671,8.333268,8.386165
Gender,8.516335,-0.666341,10.178614,12.137279,12.129884,12.071624,8.092268,8.925449,8.692231,7.993840,...,7.996544,8.095315,6.573572,5.804075,5.747478,6.852513,5.870909,4.696390,3.843216,4.145069
Legal,2.180062,12.693114,6.030221,10.719599,8.680085,8.643835,0.995655,7.085135,7.037862,6.072614,...,6.544646,6.021607,5.088183,4.431574,3.752655,5.250370,3.840253,3.493655,2.941088,3.695934
Indigenous,-1.362982,5.010737,1.658455,5.135299,3.829276,3.804415,-5.183236,2.536155,2.496676,1.899302,...,2.362035,1.764257,1.103056,0.624916,0.186659,1.209102,0.252049,-0.134309,-0.563409,0.000000
Overall,8.445907,8.187408,5.890546,-1.290110,-3.706057,-3.863788,-4.293200,-5.747026,-5.909579,-6.508398,...,-8.787384,-8.824857,-9.155938,-10.389903,-10.474640,-11.125653,-11.271704,-13.165648,-15.157989,-17.384512


### Short horizon

ARIMA

In [41]:
model_arima ='arima'
percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE, True)
percentages_prison_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,mintshrink,"case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_mean","case1_lambda_[1, 1.5]_median","case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean",wls,"case1_lambda_[1, 1.5]_mean",ols,"case2_lambda_[1, 1.5]_mean",...,"case2_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_median",bottomup,"case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_mean",erm,"case1_lambda_[0.01, 0.09]_mean",case1_lambda_1_mean,case1_lambda_1_median,"case1_lambda_[0.01, 0.09]_median"
Australia,-191.504184,-1480.878704,-1457.218166,-1328.266532,-1840.634281,-1822.407258,-471.969377,-1520.091765,-19.775671,-979.317216,...,-1734.875292,-2052.750913,-3404.582840,-2724.217620,-2655.904523,-1519.071403,-3285.395539,-3138.158949,-3336.420326,-3869.577470
State,17.439712,24.813995,22.811623,21.235931,24.630650,24.338754,14.029956,20.493138,6.734478,14.228312,...,14.015107,11.063142,12.879618,5.886495,3.877486,9.036993,7.862992,-1.271665,-2.149563,1.960978
Gender,18.102436,27.799583,25.309355,22.650049,25.124745,24.876060,14.739694,23.040468,10.023429,16.460353,...,15.801853,14.526209,13.392442,8.272774,6.837879,16.442747,9.686638,3.517781,2.475224,3.959176
Legal,-7.717306,6.488672,2.990804,1.389709,3.347128,2.913240,-11.962387,0.919062,-14.083709,-5.858706,...,-4.989760,-4.880118,-5.374847,-10.627036,-11.983740,-63.458886,-7.633453,-14.823122,-17.800227,-12.001291
Indigenous,-0.729488,8.262797,5.765253,5.309390,6.074736,5.762844,-5.263362,4.548994,-5.975455,-0.244724,...,0.635157,0.430173,0.000000,-3.498784,-4.563043,-54.737655,-1.268174,-6.481494,-8.619112,-4.119357
Overall,9.568925,8.469318,6.213484,5.488848,3.846314,3.692484,3.647937,3.593096,2.554334,1.766646,...,-4.411483,-8.553356,-19.319347,-19.425934,-20.460131,-20.821806,-22.002174,-28.135235,-31.170117,-31.923277


ETS

In [42]:
model_ets ='ets'
percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE, True)
percentages_prison_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,case2_lambda_1_median,case2_lambda_1_mean,"case1_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 5]_median",case1_lambda_1_median,"case2_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_median",...,ols,wls,"case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_mean",bottomup,"case2_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_median",erm,"case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean"
Australia,-500.521441,-454.865310,-1254.362731,-1254.362731,-1207.044828,-1325.116490,-619.351476,-1333.327511,-1510.097930,-1351.651207,...,-15.322609,-1002.874308,-2288.033893,-2288.033893,-3033.353567,-2450.290309,-2450.290309,-2921.571633,-2479.057790,-2515.981589
State,36.792202,31.798855,32.218212,32.218212,31.699840,32.390639,27.586487,31.091268,31.890350,30.612095,...,10.728614,18.853948,21.614439,21.614439,25.769501,18.790632,18.790632,15.893155,-1.613946,-1.547048
Gender,28.484293,23.892008,23.472005,23.472005,23.453355,24.244961,19.000165,22.915477,23.716982,22.725055,...,3.108864,8.698361,14.659684,14.659684,17.187082,10.946710,10.946710,8.613978,-9.019113,-9.318393
Legal,9.298158,5.645053,8.870115,8.870115,8.311559,8.848483,0.423897,7.767874,8.299436,5.149410,...,-7.167135,-6.374773,1.962699,1.962699,6.290859,-0.761176,-0.761176,-6.987274,-16.396657,-16.513560
Indigenous,2.535562,-0.494902,2.153058,2.153058,1.639178,2.153499,-4.617562,1.106614,1.490001,-2.013002,...,-10.983168,-17.938420,-3.514037,-3.514037,0.000000,-5.670920,-5.670920,-39.317612,-18.147032,-18.542188
Overall,21.781235,17.735225,12.566856,12.566856,12.553715,12.328405,11.938527,11.032826,10.395131,9.848952,...,2.815010,-0.256886,-4.086981,-4.086981,-6.138350,-8.296101,-8.296101,-18.444188,-26.953618,-27.357856


### Across 3 Samples

ARIMA

In [43]:
sample_precentages = get_results_across_samples(data_prison, model_arima, LEVELS_prison, FC_TYPE, 3, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,erm,wls,mintshrink,case1_lambda_1_mean,"case1_lambda_[0.1, 0.9]_mean","case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean","case1_lambda_[0.1, 0.9]_median",case2_lambda_1_mean,...,"case2_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median",case2_lambda_1_median,"case1_lambda_[0.01, 5]_mean",bottomup,"case2_lambda_[0.01, 5]_median","case2_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 5]_median","case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_mean"
Australia,6.791430,2.151553,-20.055026,-39.345578,-41.840461,-36.163411,-42.470952,-60.090511,-43.794971,-86.185583,...,-41.304889,-43.543002,-68.391945,-71.414366,-67.839964,-76.695404,-82.243016,-84.150750,-84.555226,-94.608589
State,-3.384196,-32.937869,-18.553872,-28.323818,-40.181286,-38.460351,-39.915826,-37.942998,-42.594195,-39.718902,...,-43.154012,-41.368197,-40.694476,-31.335875,-41.562012,-43.585759,-42.989269,-31.028340,-43.013675,-42.051214
Gender,10.092889,9.114246,8.017324,0.842039,0.318063,-0.277747,-0.933675,0.209379,-1.769626,-1.168602,...,-2.009781,-2.078927,-2.304099,0.583490,-3.267630,-1.963314,-1.975938,0.412982,-3.623027,-3.492401
Legal,8.080384,12.951586,8.074736,5.373870,1.137712,1.353184,0.496562,1.289617,0.324023,0.651236,...,-0.219727,-0.311464,-0.237788,0.899996,-1.053082,-0.063088,-0.001430,0.898131,-1.043410,-0.854885
Indigenous,6.374406,8.276488,9.253210,8.358358,1.611157,1.713222,1.185766,1.903990,0.753165,1.333966,...,0.670656,0.586078,0.547039,1.282258,0.000000,0.729423,0.838925,1.249345,0.050571,0.087997
Overall,6.800359,-0.530093,-3.903410,-10.108103,-16.316009,-16.482037,-16.847498,-17.546193,-18.119128,-18.236696,...,-19.325602,-19.386632,-19.415071,-21.338155,-21.505844,-21.866009,-22.203951,-22.329172,-22.378107,-22.707527


ETS

In [44]:
sample_precentages = get_results_across_samples(data_prison, model_ets, LEVELS_prison, FC_TYPE, 3, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,erm,case1_lambda_1_mean,ols,"case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_mean",mintshrink,case1_lambda_1_median,"case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_median","case2_lambda_[1, 1.5]_mean",...,"case2_lambda_[1, 1.5]_median","case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_mean","case1_lambda_[1, 1.5]_median","case1_lambda_[0.1, 0.9]_median","case1_lambda_[1, 1.5]_mean",case2_lambda_1_mean,"case1_lambda_[0.1, 0.9]_mean",case2_lambda_1_median,bottomup
Australia,-11.320953,-510.998199,-13.492686,-586.712528,-586.877169,-125.010396,-792.207996,-635.431331,-636.428306,-611.557807,...,-781.418343,-466.030838,-463.566486,-718.956165,-467.817298,-787.104821,-850.121134,-470.139381,-996.354740,-1495.579930
State,-1.130643,19.662699,15.328179,18.532509,18.584281,13.994524,13.632728,10.545706,11.175551,10.397753,...,9.240875,7.816168,7.766812,9.383476,5.851340,7.253616,6.826379,5.325670,5.424434,6.679349
Gender,-11.143591,16.590783,8.853649,15.974513,15.990796,10.167056,11.560161,9.083158,9.555645,9.449721,...,8.231155,6.951519,6.961621,6.986715,4.458151,5.204747,4.920100,4.612237,3.669017,5.068671
Legal,13.063626,14.461323,0.468643,10.038413,10.068541,5.649663,10.529353,8.191998,8.252352,7.702845,...,6.802746,7.584140,7.639156,6.316025,3.674707,5.017628,4.122461,3.592062,3.357641,5.302536
Indigenous,6.007655,6.639543,-3.785067,3.465955,3.487444,-0.485315,3.865886,1.997587,2.053699,1.763740,...,1.160265,1.734191,1.790738,0.774842,-1.184245,-0.162362,-0.877165,-1.201392,-1.408509,0.000000
Overall,15.681036,10.747809,5.773629,4.297061,4.284958,3.971586,1.229630,-0.614985,-0.669321,-2.340745,...,-4.271728,-4.375346,-4.389879,-4.570943,-7.515182,-7.724155,-8.897484,-9.528326,-11.540940,-12.624014


## Tourism

In [45]:
LEVELS_tourism = ['Australia', 'States', 'Regions']
data_tourism = 'tourism'

FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintsample', 'mintshrink', 'erm',
               'case1_lambda_1_median', 'case1_lambda_1_mean',
               'case1_lambda_[0.01, 0.09]_mean', 'case1_lambda_[0.01, 0.09]_median',
               'case1_lambda_[0.1, 0.9]_mean', 'case1_lambda_[0.1, 0.9]_median',
               'case1_lambda_[1, 1.5]_mean', 'case1_lambda_[1, 1.5]_median',
               'case1_lambda_[0.01, 5]_mean', 'case1_lambda_[0.01, 5]_median',
               'case2_lambda_1_median', 'case2_lambda_1_mean',
               'case2_lambda_[0.01, 0.09]_mean', 'case2_lambda_[0.01, 0.09]_median',
               'case2_lambda_[0.1, 0.9]_mean', 'case2_lambda_[0.1, 0.9]_median',
               'case2_lambda_[1, 1.5]_mean', 'case2_lambda_[1, 1.5]_median',
               'case2_lambda_[0.01, 5]_mean', 'case2_lambda_[0.01, 5]_median']

model_arima ='arima'
percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, 
                                                                   FC_TYPE)

ARIMA

In [46]:
percentages_tourism_arima ['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,case2_lambda_1_mean,case2_lambda_1_median,"case1_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_mean",ols,"case1_lambda_[0.01, 5]_median",erm,"case1_lambda_[0.01, 5]_mean","case1_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median",...,"case2_lambda_[1, 1.5]_median",case1_lambda_1_median,case1_lambda_1_mean,mintshrink,"case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_mean","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 5]_median",wls,bottomup
Australia,58.141120,55.770971,-9.017275,-13.972075,-15.403475,-21.756900,-22.529492,-25.499683,-27.575079,-28.906072,...,-38.880744,-71.665396,-72.311991,-102.149986,-131.498445,-134.183517,-155.108010,-162.650176,-190.196750,-347.466361
States,62.553958,60.375651,30.030765,31.698963,27.927216,31.665513,32.203707,30.779573,28.954761,28.275611,...,25.315688,15.434522,15.394811,11.311902,-0.451090,-1.580988,-12.180857,-14.520264,-16.443344,-76.760682
Regions,22.811185,22.595596,21.330590,21.282799,16.914354,17.488276,11.145737,17.555296,17.128917,17.059680,...,16.542120,14.908156,14.888001,21.080812,12.355440,12.188105,10.473618,10.055215,17.836383,0.000000
Overall,54.010637,52.041146,8.356078,6.310214,3.661738,1.654353,0.429077,-0.567452,-2.302717,-3.224964,...,-9.448890,-29.944319,-30.296563,-46.157768,-66.588455,-68.375778,-82.947966,-87.691027,-101.413820,-205.471997


ETS

In [47]:
model_ets ='ets'
percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE)

In [48]:
percentages_tourism_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,"case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 5]_median","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 5]_mean","case2_lambda_[1, 1.5]_mean","case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median","case2_lambda_[0.01, 0.09]_mean",...,"case2_lambda_[0.1, 0.9]_mean",erm,ols,"case2_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 5]_median","case1_lambda_[0.01, 0.09]_median",mintshrink,wls,bottomup
Australia,61.466141,60.582813,57.540521,57.719415,56.799417,56.895991,55.360825,54.427819,53.999947,53.933777,...,26.137799,34.256214,0.996985,-2.110138,-6.039028,-6.134556,-12.838277,-23.578552,-32.792912,-74.350078
States,47.800486,46.497684,48.037790,46.018601,47.257672,44.103744,43.829843,42.755838,43.609105,42.259693,...,16.437845,-36.878294,2.967490,0.234706,1.372819,-2.017444,-2.429488,-2.283716,-8.087188,-37.322787
Regions,14.812999,14.607181,15.277991,14.562816,15.231316,14.218188,14.876774,13.992330,14.079086,14.584242,...,9.348288,-111.025210,6.104559,6.617151,6.740163,6.246476,6.076872,10.001538,10.261384,0.000000
Overall,53.901337,52.989070,51.333833,50.932857,50.650877,49.910409,48.869926,47.909929,47.819411,47.520048,...,22.341279,4.279624,1.931734,-0.742528,-3.142863,-4.016806,-8.677722,-15.559769,-23.095100,-58.869128


### Across 10 Samples

ARIMA

In [49]:
sample_precentages = get_results_across_samples(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, 10)

In [50]:
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,mintsample,"case2_lambda_[0.1, 0.9]_mean","case2_lambda_[0.1, 0.9]_median","case1_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 0.09]_median",case2_lambda_1_median,mintshrink,"case2_lambda_[0.01, 0.09]_mean",case2_lambda_1_mean,...,"case1_lambda_[0.1, 0.9]_median",wls,"case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 0.09]_median",bottomup,erm
Australia,-0.102793,-2.616444,-14.387165,-17.189862,-21.478857,-19.070073,-20.130775,-21.478339,-20.893773,-21.005317,...,-26.896872,-31.323718,-37.204431,-37.589630,-51.307074,-49.842213,-51.128969,-62.350780,-73.681285,-73.351674
States,5.700418,8.244784,4.054788,3.351493,4.130343,2.792729,2.649980,3.713533,2.541716,1.628798,...,0.634003,0.386095,-3.838518,-3.681476,-5.212992,-8.691149,-9.137535,-11.675199,-16.729443,-60.020458
Regions,2.995024,2.680844,2.765872,2.675088,2.851877,2.666615,2.659094,3.647336,2.624809,2.564257,...,2.154818,2.885953,1.431611,1.441303,1.428706,1.019553,0.968226,0.460862,0.000000,-78.060715
Overall,2.084706,1.309232,-6.340889,-8.094677,-8.753371,-8.994431,-9.307939,-9.639274,-9.795645,-10.164450,...,-13.846781,-15.958033,-21.166412,-21.275100,-27.535706,-28.418343,-29.330670,-35.935557,-43.598200,-71.019554


ETS

In [52]:
sample_precentages = get_results_across_samples(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, 10)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,mintsample,ols,mintshrink,"case2_lambda_[0.01, 5]_mean",wls,"case1_lambda_[0.01, 5]_mean",case2_lambda_1_median,"case2_lambda_[0.01, 5]_median",case1_lambda_1_mean,case2_lambda_1_mean,...,case1_lambda_1_median,"case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_median","case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean",bottomup,"case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median",erm
Australia,-0.936199,-0.312004,-10.549255,-9.886269,-13.276084,-15.166252,-13.625597,-13.711052,-14.005304,-14.335930,...,-18.156356,-25.514545,-25.351502,-30.032632,-27.866813,-28.222757,-29.500052,-40.778458,-42.661582,-499.031771
States,1.366033,0.085673,-0.609660,-3.326937,-2.085833,-3.501882,-5.077241,-4.660354,-6.509241,-5.331802,...,-8.023132,-6.743695,-7.563980,-9.164834,-10.030896,-10.401946,-11.481367,-12.273678,-12.512951,-465.020886
Regions,-0.112375,1.653665,2.138080,1.163148,1.874797,0.964408,0.796416,0.976443,0.984917,0.768422,...,0.479633,0.728245,0.644299,0.469351,0.155525,0.077250,0.000000,-0.053899,-0.047084,-430.547789
Overall,1.244626,0.714369,-4.469238,-5.246765,-6.489922,-7.275679,-7.357482,-7.553463,-7.661851,-7.794588,...,-10.864405,-13.075485,-13.454144,-15.963462,-16.092584,-16.447716,-18.202915,-22.233864,-23.081646,-442.101289


### Short Horizon

ARIMA

In [53]:
percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, True)

In [54]:
percentages_tourism_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,case2_lambda_1_mean,case2_lambda_1_median,"case1_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_mean",ols,mintsample,"case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean",erm,"case1_lambda_[1, 1.5]_mean",...,"case1_lambda_[0.1, 0.9]_mean",case1_lambda_1_mean,case1_lambda_1_median,mintshrink,"case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.1, 0.9]_mean","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 5]_median",wls,bottomup
Australia,54.988731,50.068162,-11.651108,-15.728280,-14.062558,-22.866144,-19.212785,-20.685921,-26.212028,-27.031475,...,-40.992861,-60.904546,-61.042456,-84.406279,-111.111157,-112.338494,-130.696435,-137.960899,-158.864550,-280.031133
States,66.438605,63.396634,31.878049,32.116394,29.298807,38.741781,34.779766,34.496689,34.610867,31.938425,...,26.848192,20.122211,20.077036,15.946678,4.370039,3.519061,-6.158471,-8.709467,-10.003305,-62.086668
Regions,18.924837,18.540637,17.348972,17.164748,12.922287,21.530602,14.086891,14.245252,9.427498,13.876614,...,13.113725,12.112019,12.046904,17.557013,9.798562,9.707773,8.290616,7.858344,15.250644,0.000000
Overall,52.987451,49.317301,5.827323,3.636704,3.061019,2.377803,2.063598,1.194968,-2.526909,-3.108240,...,-12.406801,-25.494755,-25.593761,-38.809135,-58.100234,-59.042667,-72.229989,-77.044235,-87.773481,-172.155473


ETS

In [55]:
model_ets ='ets'
percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, True)
percentages_tourism_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,"case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_mean",erm,"case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 5]_median","case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_mean",...,"case2_lambda_[0.1, 0.9]_mean",mintsample,ols,"case2_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 5]_median",mintshrink,"case1_lambda_[0.01, 0.09]_median",wls,bottomup
Australia,57.763669,56.781120,52.808048,52.046311,78.937392,50.037334,49.519556,48.492736,48.940042,49.054036,...,20.205753,7.617090,2.487777,-8.396444,-16.053506,-17.298986,-20.707002,-26.095451,-29.676360,-75.219439
States,63.736149,63.158731,61.726981,60.632357,27.476677,60.088177,59.364956,60.917710,59.873644,59.200065,...,38.319908,42.280236,7.375898,24.258221,22.808561,20.302068,21.730145,17.859364,16.004130,-5.809304
Regions,12.605384,12.548155,12.239777,12.123516,-85.805523,12.849947,12.659214,12.416790,11.869261,11.901861,...,8.150173,-0.727899,1.256380,5.620504,5.271973,4.831181,8.877108,4.396891,9.847346,0.000000
Overall,54.749565,53.962348,51.000084,50.218758,49.394179,48.854865,48.317849,48.022826,47.992785,47.899199,...,23.584059,15.557691,3.602071,1.277016,-4.075705,-5.559827,-6.996159,-11.909284,-14.147982,-50.104556


### Across 10 Samples

ARIMA

In [56]:
sample_precentages = get_results_across_samples(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, 10, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,mintsample,ols,"case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.1, 0.9]_mean","case2_lambda_[0.01, 0.09]_mean",case1_lambda_1_median,"case2_lambda_[0.1, 0.9]_median",case1_lambda_1_mean,mintshrink,"case1_lambda_[0.01, 5]_mean",...,case2_lambda_1_mean,"case1_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median",case2_lambda_1_median,"case2_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 0.09]_median",bottomup,erm
Australia,6.941853,1.309730,-0.832781,-1.845082,-4.865044,-3.485032,-6.183705,-8.096947,-7.830944,-15.417940,...,-19.782341,-18.050428,-19.330538,-26.005493,-31.981778,-37.481030,-33.725576,-46.539388,-46.781020,-85.078352
States,5.308397,4.597570,2.207042,3.806142,1.917389,1.285421,2.881958,0.369784,2.485396,1.479175,...,-3.926292,-3.259350,-4.075688,-5.088446,-8.205814,-5.145519,-9.222992,-12.244380,-14.053088,-76.924334
Regions,1.388308,1.618156,1.890027,1.846980,1.759731,1.223933,1.689613,0.972000,2.559926,1.490221,...,1.026978,0.722486,0.628778,0.757775,0.417530,0.751422,0.241514,-0.364121,0.000000,-85.273452
Overall,4.874336,1.915430,0.393047,-0.553987,-1.535595,-1.805116,-3.141669,-3.721890,-3.741096,-5.989355,...,-11.618376,-11.649274,-12.617250,-14.702987,-18.747279,-20.185541,-20.195610,-27.918349,-28.222987,-86.123388


ETS

In [57]:
sample_precentages = get_results_across_samples(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, 10, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,mintsample,ols,mintshrink,wls,case2_lambda_1_median,case2_lambda_1_mean,"case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean","case2_lambda_[0.1, 0.9]_mean",case1_lambda_1_mean,...,case1_lambda_1_median,"case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_median","case1_lambda_[1, 1.5]_median",bottomup,"case1_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 0.09]_median","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median",erm
Australia,2.256249,0.320700,-8.599460,-11.374274,-12.014013,-13.584239,-16.542600,-17.610564,-17.831106,-20.330750,...,-24.279091,-30.703441,-34.064717,-33.702330,-31.633376,-35.084392,-35.489373,-41.393337,-43.491268,-691.212052
States,1.549397,0.377122,1.965272,0.694810,-0.802669,-1.390700,0.709850,0.546250,-1.141185,-3.143027,...,-4.338498,-6.382088,-6.043473,-6.338171,-7.609484,-7.562151,-8.456049,-6.240082,-6.510238,-463.297752
Regions,-2.362432,0.821203,1.208716,0.954388,0.626785,0.600772,0.614460,0.599986,0.469973,0.934291,...,0.418398,0.027361,0.370345,0.149695,0.000000,0.045230,-0.102731,0.315498,0.333359,-373.395147
Overall,1.032401,0.947712,-2.447539,-4.256308,-4.462591,-5.349332,-6.160362,-6.529563,-7.730506,-7.909637,...,-10.538753,-14.859153,-15.281320,-15.787851,-15.945978,-16.847725,-17.303571,-18.207367,-19.048391,-505.913517


## Wikipedia

In [58]:
LEVELS_wiki = ['Total', 'Language', 'Access', 'Agent', 'Article']

data_wiki = 'wikipedia'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm',
               'case1_lambda_1_median', 'case1_lambda_1_mean',
               'case1_lambda_[0.01, 0.09]_mean', 'case1_lambda_[0.01, 0.09]_median',
               'case1_lambda_[0.1, 0.9]_mean', 'case1_lambda_[0.1, 0.9]_median',
               'case1_lambda_[1, 1.5]_mean', 'case1_lambda_[1, 1.5]_median',
               'case1_lambda_[0.01, 5]_mean', 'case1_lambda_[0.01, 5]_median',
               'case2_lambda_1_median', 'case2_lambda_1_mean',
               'case2_lambda_[0.01, 0.09]_mean', 'case2_lambda_[0.01, 0.09]_median',
               'case2_lambda_[0.1, 0.9]_mean', 'case2_lambda_[0.1, 0.9]_median',
               'case2_lambda_[1, 1.5]_mean', 'case2_lambda_[1, 1.5]_median',
               'case2_lambda_[0.01, 5]_mean', 'case2_lambda_[0.01, 5]_median']


model_arima ='arima'
percentages_wiki_arima = get_results(data_wiki, model_arima, LEVELS_wiki, FC_TYPE)


ARIMA

In [59]:
percentages_wiki_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,erm,"case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 5]_median","case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_mean",...,case2_lambda_1_mean,case2_lambda_1_median,"case2_lambda_[0.1, 0.9]_median","case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 0.09]_median"
Total,5.558230,63.533055,26.546894,-576.149974,-148.868118,52.470260,70.576875,70.558313,71.587685,68.739813,...,52.793785,52.051280,56.218151,50.219661,50.210920,49.851622,49.812982,42.055927,41.871990,28.962341
Language,4.078147,49.086221,27.680232,-367.802124,-234.746963,59.660010,65.001812,64.960132,59.462991,56.226195,...,48.182294,47.475758,46.636527,42.462015,42.455937,42.219334,42.190812,39.182825,39.070655,31.361464
Access,-15.883146,-181.629284,7.762264,-380.261773,-708.830221,43.136374,47.576887,47.522685,39.414455,35.878557,...,28.895422,28.118183,26.291125,21.815907,21.811597,21.552801,21.525304,20.508332,20.455407,13.567972
Agent,-0.533560,-218.037269,-1175.854866,-678.567526,-2224.551740,-5048.583604,-5200.676428,-5200.795429,-5344.860575,-5481.555249,...,-5634.137135,-5645.824487,-5689.095512,-5769.203329,-5769.323592,-5774.202153,-5774.770166,-5779.173624,-5780.038655,-5878.402995
Article,-22.309463,-604.349322,0.000000,-434.994508,-867.911173,-1462.231872,-1462.093488,-1462.102273,-1462.202707,-1462.180708,...,-1462.227900,-1462.229460,-1462.239685,-1462.286722,-1462.286807,-1462.289084,-1462.289786,-1462.287860,-1462.288898,-1462.342650
Overall,-1.555836,-83.177893,-138.239392,-491.482962,-601.094699,-788.665973,-800.419143,-800.461054,-821.825822,-842.336702,...,-871.352205,-873.462785,-878.215395,-892.710122,-892.731416,-893.605324,-893.705907,-897.979393,-898.196835,-918.838171


ETS

In [60]:
model_ets ='ets'
percentages_wiki_ets = get_results(data_wiki, model_ets, LEVELS_wiki, FC_TYPE)
percentages_wiki_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,"case1_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 5]_median",case1_lambda_1_mean,"case1_lambda_[0.01, 0.09]_mean",case1_lambda_1_median,"case1_lambda_[0.01, 0.09]_median",...,case2_lambda_1_median,"case2_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_mean","case2_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_median","case2_lambda_[1, 1.5]_mean","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 5]_median","case2_lambda_[1, 1.5]_median",erm
Total,-1.780968,-9.234173,-53.404524,-2198.788272,-178.798598,-165.616310,-6.673727,-89.172561,-9.110571,-104.826662,...,33.212800,-31.966685,-31.852000,-21.086326,-20.951824,-9.670777,-19.881098,-23.546309,-36.184243,-7392.688084
Language,-4.896559,-14.541256,-45.097026,-1237.700892,-128.966999,-125.679347,-24.020533,-48.574835,-26.021116,-56.494372,...,12.660231,-29.616510,-29.580377,-26.996562,-26.943218,-24.035246,-26.641616,-29.136869,-37.951497,-6595.797020
Access,17.444289,-155.538459,6.347685,-239.992403,2.528017,1.839674,19.607580,8.346688,17.861557,6.828836,...,19.187023,10.885722,10.891338,11.023310,11.034110,10.865719,11.444647,10.913201,7.825578,-1611.312745
Agent,-1.623400,-183.490835,-1206.884441,-475.183814,-4539.497777,-4660.448504,-4848.767514,-4859.366247,-4935.906685,-4857.131412,...,-5109.340027,-5042.568763,-5043.059868,-5091.481458,-5091.919537,-5159.104436,-5164.525102,-5169.768919,-5310.916965,-5221.273236
Article,-19.885314,-618.562267,0.000000,-356.264750,-1291.846808,-1291.847101,-1291.701574,-1291.813457,-1291.715108,-1291.827451,...,-1291.701844,-1291.782241,-1291.781651,-1291.761862,-1291.761118,-1291.765569,-1291.764401,-1291.765415,-1291.783065,-3937.032444
Overall,-0.401148,-218.159124,-297.392384,-726.245379,-1381.790358,-1408.340472,-1412.089343,-1432.640648,-1433.827198,-1435.789701,...,-1463.449972,-1464.759448,-1464.853147,-1474.425319,-1474.499895,-1488.489825,-1491.383593,-1493.621048,-1530.861582,-4415.041896


### Across 10 Samples


ARIMA

In [61]:
sample_precentages = get_results_across_samples(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, 10)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,"case1_lambda_[1, 1.5]_mean","case2_lambda_[0.1, 0.9]_mean","case1_lambda_[0.01, 5]_mean","case1_lambda_[1, 1.5]_median","case1_lambda_[0.01, 5]_median",case1_lambda_1_median,...,case1_lambda_1_mean,"case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median","case2_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_mean",erm
Total,3.422901,-49.069284,-83.588490,-2830.477559,6.791608,-51.044737,-37.244501,14.447520,-39.095794,3.090897,...,-2.874726,-48.904859,-55.933575,-44.370638,-48.123657,-1719.214923,-1770.917921,-3937.554164,-4824.213156,-14359.095402
Language,-1.622363,-3.071049,0.491071,-1857.835748,8.208219,19.338339,27.157329,28.193424,26.371138,30.973342,...,29.478934,19.838519,12.435849,30.196913,22.794010,-1176.779817,-1228.900788,-2879.438805,-3464.579967,-6111.712556
Access,-5.337866,-200.825003,4.555721,-855.903947,7.433951,9.891386,11.722371,13.770729,12.103550,18.997681,...,18.285751,10.128134,7.959031,14.089801,11.599344,-236.470029,-240.891587,-618.205332,-910.097880,-3889.995282
Agent,1.247197,-410.233107,-1805.800444,-1367.642395,-7070.362473,-7055.755136,-7107.893586,-7189.532380,-7122.066316,-7153.986704,...,-7185.495066,-7205.020138,-7206.830793,-7259.852321,-7452.101906,-6638.633575,-6855.746258,-7072.332777,-6690.185147,-5042.310741
Article,-37.584043,-1021.000137,0.000000,-871.800200,-3113.330188,-3113.296492,-3113.317053,-3113.236883,-3113.292964,-3113.201531,...,-3113.204770,-3113.287069,-3113.361695,-3113.262085,-3113.259835,-3117.412403,-3117.350148,-3123.236265,-3128.012455,-4343.768238
Overall,-2.268458,-238.358392,-348.506173,-1419.983236,-1710.347241,-1729.595347,-1732.342685,-1733.866478,-1735.009975,-1736.389600,...,-1743.528880,-1751.595700,-1753.929104,-1757.003496,-1792.768371,-2263.415538,-2303.247121,-3200.768276,-3653.921565,-4734.783254


ETS

In [62]:
sample_precentages = get_results_across_samples(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, 10)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,"case1_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 5]_median",case2_lambda_1_mean,case2_lambda_1_median,"case2_lambda_[0.01, 5]_mean",case1_lambda_1_mean,...,"case2_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median",erm
Total,-1.545956,-156.784548,-166.445853,-3064.939061,-577.317521,-600.120773,-231.916519,-233.756658,-471.546012,-250.974005,...,-273.924399,-239.916297,-267.545713,-212.437927,-206.827522,-217.826108,-217.899530,-216.567458,-221.748717,-17421.643250
Language,0.354273,-55.236804,-112.271289,-1074.965778,-410.831450,-418.991696,-167.037681,-160.981659,-250.474757,-201.644441,...,-143.276032,-129.421396,-140.717692,-113.489613,-113.586837,-117.361664,-119.955344,-121.390001,-124.922234,-20042.309418
Access,4.350991,-325.506582,-65.638359,-584.039207,-238.890588,-241.139735,-114.170385,-110.524654,-121.773062,-128.129305,...,-77.199561,-69.306477,-76.567967,-64.814025,-64.883006,-66.198451,-66.767133,-69.856525,-70.887024,-19000.820719
Agent,18.459044,-162.825540,-1163.388061,-379.813470,-4585.153335,-4593.434327,-4541.683981,-4564.387185,-4702.729010,-4560.429790,...,-4766.984545,-4812.561409,-4770.006781,-4820.832474,-4836.804022,-4830.270707,-4844.769779,-4909.266692,-4921.081061,-12120.949375
Article,-34.015120,-1180.886799,0.000000,-902.009761,-2731.285093,-2731.298250,-2731.175865,-2731.143454,-2730.944234,-2731.294762,...,-2730.801258,-2730.692971,-2730.793844,-2730.679837,-2730.683843,-2730.692395,-2730.693662,-2730.719449,-2730.726410,-5464.913304
Overall,3.145778,-264.731826,-385.198790,-715.006541,-1787.663666,-1790.761110,-1798.496509,-1803.305712,-1810.096712,-1814.755423,...,-1834.521924,-1834.602761,-1835.246626,-1837.364468,-1841.174769,-1842.187424,-1846.092650,-1853.988607,-1857.406735,-9819.259100


### Short Horizon

ARIMA

In [63]:
model_arima ='arima'
percentages_wiki_arima = get_results(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, True)

percentages_wiki_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,erm,"case2_lambda_[0.01, 5]_median","case2_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 0.09]_mean","case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_mean",...,"case2_lambda_[0.1, 0.9]_median",case2_lambda_1_mean,case2_lambda_1_median,"case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_median","case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 0.09]_median"
Total,7.483053,88.750981,42.250895,-140.649044,-32.856971,85.817211,85.740159,64.241252,87.095285,81.815793,...,70.359991,64.745722,63.935655,64.462084,64.450282,64.281189,64.248602,55.046027,54.889410,44.382082
Language,5.416017,67.080294,44.226367,-52.195351,-80.616611,84.899133,84.844194,77.457121,76.568943,72.389622,...,63.505009,63.264410,62.448853,59.076650,59.067947,58.954687,58.927171,54.491859,54.388881,48.026399
Access,-27.344920,-105.804044,22.923339,-103.063679,-355.994491,80.084266,80.031601,75.481780,64.383050,58.952882,...,47.511727,48.898747,47.741541,41.289500,41.280861,41.106217,41.072280,37.807512,37.742090,30.564631
Agent,-4.590453,-69.041907,-706.803027,-290.462032,-682.401860,-3707.391702,-3708.535843,-3683.377194,-3929.196357,-4016.435091,...,-4163.657460,-4148.040151,-4160.676723,-4235.456688,-4235.547574,-4237.344007,-4237.716016,-4268.964892,-4269.653706,-4341.505727
Article,-77.840614,-2032.131377,0.000000,-1392.418632,-2264.924709,-4808.298024,-4808.273025,-4808.826661,-4808.900452,-4808.901538,...,-4809.217517,-4809.204165,-4809.225218,-4809.465907,-4809.466200,-4809.481896,-4809.479397,-4809.537631,-4809.541715,-4809.779319
Overall,-0.577058,-6.218686,-47.409057,-153.314907,-210.886701,-463.938045,-464.127015,-473.284002,-493.670915,-508.031096,...,-534.208947,-534.707779,-536.917887,-547.217016,-547.236405,-547.574312,-547.644138,-557.001110,-557.188508,-572.898172


ETS

In [64]:
model_ets ='ets'
percentages_wiki_ets = get_results(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, True)
percentages_wiki_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,"case1_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_median",case2_lambda_1_mean,"case1_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_median",...,"case1_lambda_[0.1, 0.9]_median",case1_lambda_1_median,"case1_lambda_[0.01, 5]_median",case2_lambda_1_median,"case1_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_median","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 5]_median",erm
Total,12.151204,76.479472,30.725135,-1958.245839,81.583611,79.518714,74.911790,63.098273,75.163173,71.852065,...,64.852072,56.383565,38.058023,42.386290,38.963350,38.109634,38.529276,32.878108,32.876982,-9568.214846
Language,32.703180,67.375540,48.303821,-1085.779314,80.614000,79.624636,78.644916,73.561928,75.814102,72.900577,...,67.546802,68.204421,56.772971,55.754234,53.765921,52.954134,52.386442,49.511720,49.511910,-8615.357842
Access,-21.729964,-90.103783,45.023371,-571.427111,61.164971,60.717100,62.303116,56.312647,58.637012,57.074068,...,54.286843,54.914803,49.410676,49.148280,47.861998,47.465756,47.029763,45.622319,45.622375,-5227.782933
Agent,-2.520539,-190.607762,-2432.122122,-949.462596,-10964.743683,-10968.211126,-11033.192856,-11020.135314,-11076.199934,-11142.292179,...,-11242.137995,-11262.384739,-11329.997082,-11441.441077,-11458.869702,-11481.728120,-11490.104994,-11527.312146,-11527.311180,-13640.656626
Article,-117.100117,-3300.863518,0.000000,-2254.682476,-7861.554123,-7861.581121,-7861.470778,-7861.622198,-7861.582103,-7861.584038,...,-7861.600469,-7861.645237,-7861.710961,-7861.720516,-7861.743168,-7861.751681,-7861.755759,-7861.781924,-7861.781954,-24529.789717
Overall,-2.437618,-257.807025,-481.804726,-1264.918343,-2810.257832,-2811.872553,-2826.651504,-2829.566834,-2837.083244,-2852.886028,...,-2877.613020,-2883.849782,-2906.866125,-2929.425663,-2934.753978,-2940.064617,-2941.940765,-2952.259736,-2952.259778,-10341.067784


### Across 10 Samples

ARIMA

In [65]:
sample_precentages = get_results_across_samples(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, 10, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,"case1_lambda_[0.01, 5]_mean","case1_lambda_[1, 1.5]_mean",case2_lambda_1_median,"case1_lambda_[0.01, 5]_median","case2_lambda_[0.1, 0.9]_mean",case2_lambda_1_mean,...,case1_lambda_1_mean,"case2_lambda_[1, 1.5]_mean","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median","case2_lambda_[1, 1.5]_median","case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_mean",erm
Total,-0.869935,44.925779,-133.829749,-1561.729166,-64.566879,10.231841,-12.441800,-68.480405,-83.723178,-14.210637,...,-23.058217,-84.127022,-89.471728,-93.149770,-88.141317,-961.530482,-1012.008612,-2261.860071,-3053.299109,-15342.451618
Language,3.822773,51.618391,19.723810,-1357.105422,41.125400,27.307303,49.481540,38.041726,41.276150,49.574952,...,47.138951,44.242154,36.864965,33.049095,41.861926,-829.257596,-859.424776,-2164.611978,-2760.563042,-7023.325979
Access,-18.796859,-309.460023,18.977092,-705.319729,24.705183,26.648083,32.853038,24.763342,29.107700,32.970278,...,31.668031,27.277382,23.460347,22.329235,27.214233,-106.068061,-123.895161,-335.690189,-635.614418,-4724.550907
Agent,1.726982,-379.851589,-1772.142320,-1136.338249,-7124.582966,-7271.436308,-7197.962272,-7142.373757,-7208.855393,-7231.958131,...,-7276.741026,-7353.351409,-7406.486562,-7418.520350,-7550.171998,-7149.868895,-7060.859229,-7282.170468,-7110.502169,-5602.992375
Article,-69.474652,-2014.868419,0.000000,-1579.995772,-5844.615211,-5844.504784,-5844.359131,-5844.594994,-5844.369504,-5844.362464,...,-5844.396917,-5844.562486,-5844.520737,-5844.652807,-5844.489831,-5847.422755,-5847.906350,-5852.424160,-5860.099557,-6731.175050
Overall,-1.498970,-200.027938,-276.173385,-1023.755085,-1536.605357,-1540.566934,-1541.427946,-1542.151877,-1543.878272,-1547.314587,...,-1555.996206,-1566.130063,-1575.824910,-1578.642584,-1597.049741,-1860.680061,-1877.059268,-2432.139567,-2906.110653,-4704.861654


ETS

In [66]:
ample_precentages = get_results_across_samples(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, 10, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,wls,bottomup,mintshrink,"case1_lambda_[0.01, 5]_mean","case1_lambda_[1, 1.5]_mean",case2_lambda_1_median,"case1_lambda_[0.01, 5]_median","case2_lambda_[0.1, 0.9]_mean",case2_lambda_1_mean,...,case1_lambda_1_mean,"case2_lambda_[1, 1.5]_mean","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median","case2_lambda_[1, 1.5]_median","case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_mean",erm
Total,-0.869935,44.925779,-133.829749,-1561.729166,-64.566879,10.231841,-12.441800,-68.480405,-83.723178,-14.210637,...,-23.058217,-84.127022,-89.471728,-93.149770,-88.141317,-961.530482,-1012.008612,-2261.860071,-3053.299109,-15342.451618
Language,3.822773,51.618391,19.723810,-1357.105422,41.125400,27.307303,49.481540,38.041726,41.276150,49.574952,...,47.138951,44.242154,36.864965,33.049095,41.861926,-829.257596,-859.424776,-2164.611978,-2760.563042,-7023.325979
Access,-18.796859,-309.460023,18.977092,-705.319729,24.705183,26.648083,32.853038,24.763342,29.107700,32.970278,...,31.668031,27.277382,23.460347,22.329235,27.214233,-106.068061,-123.895161,-335.690189,-635.614418,-4724.550907
Agent,1.726982,-379.851589,-1772.142320,-1136.338249,-7124.582966,-7271.436308,-7197.962272,-7142.373757,-7208.855393,-7231.958131,...,-7276.741026,-7353.351409,-7406.486562,-7418.520350,-7550.171998,-7149.868895,-7060.859229,-7282.170468,-7110.502169,-5602.992375
Article,-69.474652,-2014.868419,0.000000,-1579.995772,-5844.615211,-5844.504784,-5844.359131,-5844.594994,-5844.369504,-5844.362464,...,-5844.396917,-5844.562486,-5844.520737,-5844.652807,-5844.489831,-5847.422755,-5847.906350,-5852.424160,-5860.099557,-6731.175050
Overall,-1.498970,-200.027938,-276.173385,-1023.755085,-1536.605357,-1540.566934,-1541.427946,-1542.151877,-1543.878272,-1547.314587,...,-1555.996206,-1566.130063,-1575.824910,-1578.642584,-1597.049741,-1860.680061,-1877.059268,-2432.139567,-2906.110653,-4704.861654


## Labour Data

In [67]:
LEVELS_labour = ['Total Employees', 'Main Occupation', 'Employment Status', 'Gender']

data_labour = 'labour'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'mintsample','erm',
               'case1_lambda_1_median', 'case1_lambda_1_mean',
               'case1_lambda_[0.01, 0.09]_mean', 'case1_lambda_[0.01, 0.09]_median',
               'case1_lambda_[0.1, 0.9]_mean', 'case1_lambda_[0.1, 0.9]_median',
               'case1_lambda_[1, 1.5]_mean', 'case1_lambda_[1, 1.5]_median',
               'case1_lambda_[0.01, 5]_mean', 'case1_lambda_[0.01, 5]_median',
               'case2_lambda_1_median', 'case2_lambda_1_mean',
               'case2_lambda_[0.01, 0.09]_mean', 'case2_lambda_[0.01, 0.09]_median',
               'case2_lambda_[0.1, 0.9]_mean', 'case2_lambda_[0.1, 0.9]_median',
               'case2_lambda_[1, 1.5]_mean', 'case2_lambda_[1, 1.5]_median',
               'case2_lambda_[0.01, 5]_mean', 'case2_lambda_[0.01, 5]_median']

ARIMA

In [68]:
model_arima ='arima'
percentages_labour_arima = get_results(data_labour, model_arima, LEVELS_labour, FC_TYPE)

percentages_labour_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,mintsample,mintshrink,ols,"case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean",wls,case1_lambda_1_mean,case1_lambda_1_median,bottomup,"case2_lambda_[0.01, 5]_mean",...,"case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 5]_median","case1_lambda_[0.01, 0.09]_mean",case2_lambda_1_median,"case1_lambda_[1, 1.5]_median",case2_lambda_1_mean,"case2_lambda_[0.1, 0.9]_median",erm,"case2_lambda_[0.1, 0.9]_mean"
Total Employees,14.953236,-11.653076,-5.150195,-23.186437,1.113312,-34.669487,-62.583717,-68.374181,-88.715285,-81.268733,...,-108.040846,-115.983264,-78.057563,-138.759149,-130.774979,-71.435461,-150.643315,-220.907636,0.478694,-253.921968
Main Occupation,34.670086,22.939207,14.924121,23.198672,17.354654,17.425253,22.395836,22.107407,16.785035,12.830593,...,7.435511,7.768291,-0.377432,11.653945,8.561774,-12.449238,5.025796,11.218283,-88.651984,6.238045
Employment Status,24.530701,8.593970,-2.341733,0.613801,-9.413386,0.894809,3.184757,2.674526,0.467585,-6.752568,...,-14.101422,-10.465834,-28.645298,-6.565230,-10.012718,-35.953790,-12.676582,-4.377327,-42.506073,-8.696016
Gender,6.705767,2.799196,-1.890739,0.451152,-6.941596,-1.499057,1.927652,2.181937,0.000000,-4.868346,...,-8.329916,-6.268969,-20.084489,-5.108277,-6.486342,-22.498154,-8.194570,-4.525880,-50.623411,-6.022886
Overall,22.270409,7.621977,2.742903,2.611187,2.143177,-2.144579,-5.529244,-7.003335,-14.283412,-16.702632,...,-27.020570,-27.357565,-28.860382,-29.963848,-30.337275,-33.661688,-36.956674,-47.832301,-49.473138,-58.254631


ETS

In [69]:
model_ets ='ets'
percentages_labour_ets= get_results(data_labour,model_ets, LEVELS_labour, FC_TYPE)

percentages_labour_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,mintsample,ols,mintshrink,wls,"case2_lambda_[0.01, 5]_mean","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 5]_median","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.1, 0.9]_median",case2_lambda_1_mean,...,"case2_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean",bottomup,"case2_lambda_[0.1, 0.9]_mean","case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean",erm
Total Employees,-6.873949,-15.294247,-45.502673,-85.678619,-114.018789,-129.600515,-125.575871,-129.525985,-131.974217,-137.397079,...,-146.733281,-146.804210,-146.781403,-148.714531,-167.464575,-163.294583,-173.311644,-242.510365,-274.127485,-394.551986
Main Occupation,25.638451,15.137206,16.544675,11.690180,11.024343,10.296266,9.399723,10.146888,11.307607,9.621853,...,8.572337,8.558230,8.540526,8.405533,9.393638,7.848133,6.703181,5.262794,5.996404,-96.985615
Employment Status,19.388811,2.580386,7.727508,0.654435,-3.216862,-2.847619,-4.385650,-3.177351,-7.178673,-3.705353,...,-4.332490,-4.335364,-4.349731,-4.428752,-5.210339,-6.166489,-7.247637,-8.639120,-5.890635,-107.351082
Gender,11.182820,4.562546,5.464205,1.492789,0.669697,0.866117,0.000000,0.669697,-1.088726,0.370391,...,0.056120,0.065473,0.054249,0.000000,0.138429,-1.113044,-1.704173,-2.566549,0.093533,-84.439830
Overall,14.298780,3.382068,-1.037650,-13.761202,-21.137928,-24.543684,-24.558160,-24.701602,-26.207741,-26.736765,...,-29.285818,-29.304899,-29.311479,-29.797076,-33.607496,-33.733830,-36.630960,-52.280593,-57.492532,-160.130414


### Across 5 Samples


ARIMA

In [70]:
sample_precentages = get_results_across_samples(data_labour, model_arima, LEVELS_labour, FC_TYPE, 5)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,mintshrink,wls,case2_lambda_1_mean,"case2_lambda_[1, 1.5]_mean","case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median",case2_lambda_1_median,case1_lambda_1_median,case1_lambda_1_mean,...,"case1_lambda_[0.01, 0.09]_mean",bottomup,"case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 0.09]_median","case2_lambda_[0.1, 0.9]_median","case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 0.09]_median",mintsample,erm
Total Employees,-2.720414,-50.594389,-71.615074,-134.135461,-145.056390,-140.828900,-152.489612,-135.647562,-153.936770,-162.887122,...,-181.817281,-188.420876,-185.594548,-190.587491,-207.129945,-179.742153,-199.816231,-187.430794,-588.908578,-962.763777
Main Occupation,8.011827,1.877739,0.445173,-8.872352,-9.554847,-16.276704,-10.024037,-10.734952,-10.912147,-11.190995,...,-16.364286,-14.615270,-17.877587,-18.180033,-19.007225,-24.223010,-20.173310,-25.514085,-66.986899,-522.412762
Employment Status,7.488523,8.334446,6.060285,-5.453886,-5.628772,-7.882300,-5.177170,-6.602591,-5.664872,-5.860806,...,-9.026711,-7.722610,-9.060777,-9.936089,-9.854921,-13.784445,-11.038488,-14.358564,-50.472192,-483.101377
Gender,7.816864,8.680005,7.905683,1.348597,0.988617,-1.506819,1.451571,0.427395,1.286145,1.154379,...,-0.689757,0.000000,-0.978817,-1.252922,-1.597183,-5.030161,-1.862406,-5.483256,-68.269732,-548.241020
Overall,7.063768,-0.694001,-6.009772,-25.533314,-25.590842,-25.927840,-26.957103,-27.627101,-28.210488,-29.642286,...,-35.181030,-36.374657,-37.541203,-37.713003,-39.936972,-40.329882,-41.331480,-42.276596,-87.119945,-549.105099


ETS

In [71]:
sample_precentages = get_results_across_samples(data_labour, model_ets, LEVELS_labour, FC_TYPE, 5)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,mintshrink,case2_lambda_1_median,case2_lambda_1_mean,"case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.01, 5]_mean",wls,"case1_lambda_[0.01, 5]_median","case2_lambda_[0.01, 0.09]_mean",...,"case2_lambda_[0.1, 0.9]_mean","case2_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median",case1_lambda_1_median,"case2_lambda_[0.01, 5]_median",bottomup,"case2_lambda_[0.01, 5]_mean",case1_lambda_1_mean,mintsample,erm
Total Employees,-13.890569,-69.303613,-118.045237,-126.482758,-126.283256,-120.978017,-146.538083,-114.650325,-147.126760,-126.014986,...,-155.108647,-161.360932,-163.702370,-167.264297,-170.973496,-174.261772,-190.879979,-185.987643,-96.513502,-4234.620006
Main Occupation,9.612300,12.411999,16.526619,15.091165,12.641978,12.204362,16.176025,12.532165,15.900760,13.410387,...,10.225234,13.014236,13.179798,12.432833,11.862838,11.819814,10.859621,11.111837,-141.102725,-3351.800041
Employment Status,1.656005,3.692503,7.474049,6.320041,5.155375,4.741126,7.144239,2.960811,6.572318,5.293775,...,3.528579,3.865003,4.081016,3.028738,3.398466,3.214186,2.544205,2.399178,-151.201223,-3270.022069
Gender,-2.426612,-1.447356,2.955225,2.241970,1.762992,1.515710,3.009426,-1.471479,2.493923,1.972680,...,0.938032,0.512664,0.619952,-0.262727,0.124785,0.000000,-0.420082,-0.868975,-158.064297,-3173.978197
Overall,7.708044,3.808258,0.603291,-1.580198,-1.704258,-1.884379,-2.330029,-2.344520,-2.753859,-3.107603,...,-7.501254,-8.593305,-8.646439,-8.984580,-9.221664,-9.557841,-12.355988,-12.533750,-125.908898,-3225.891144


### Short horizon

ARIMA

In [72]:
model_arima ='arima'
percentages_labour_arima = get_results(data_labour,model_arima, LEVELS_labour, FC_TYPE,
                                                                True)

percentages_labour_arima['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,case1_lambda_1_median,case1_lambda_1_mean,"case2_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_median","case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 0.09]_mean",bottomup,"case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_mean","case1_lambda_[0.01, 0.09]_median",...,"case2_lambda_[0.1, 0.9]_mean",mintshrink,"case1_lambda_[1, 1.5]_median",ols,"case2_lambda_[0.01, 5]_median","case1_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 5]_median",mintsample,"case1_lambda_[0.01, 5]_mean",erm
Total Employees,64.595757,63.609693,64.115766,62.121747,61.571081,59.487063,54.388774,56.699680,56.678464,68.641362,...,-11.900602,34.077509,53.583518,18.795075,56.698328,-6.475967,-2.160773,27.000877,-290.061661,67.837862
Main Occupation,35.101627,33.895446,28.318789,27.696955,26.212383,25.583934,27.903682,27.369500,26.607644,25.137956,...,27.206324,16.486309,5.242587,8.149990,6.164312,10.726630,13.764684,4.867171,-21.238265,-166.857590
Employment Status,13.047844,12.040162,11.278050,11.038529,7.223129,9.077602,10.929656,10.005444,8.780016,2.892397,...,8.778806,1.981492,-8.998972,-2.106091,-21.290752,-2.303272,-6.784008,-12.716385,-42.456904,-83.041190
Gender,1.697634,0.048417,0.432730,0.226956,3.062398,2.780972,0.000000,-0.750469,-1.579616,-7.577316,...,-1.388973,-13.233069,-10.815227,-8.152273,-21.582037,-7.680203,-10.733523,-34.612359,-33.958724,-85.834897
Overall,25.767475,24.551734,22.731240,22.095314,21.086561,21.020474,20.824709,20.587795,19.796005,18.175017,...,8.927903,8.262051,5.232051,3.133495,-0.017457,-0.086037,-0.427691,-5.770717,-73.838500,-85.872466


ETS

In [73]:
model_ets ='ets'
percentages_labour_ets= get_results(data_labour, model_ets, LEVELS_labour, FC_TYPE,
                                                           True)

percentages_labour_ets['MSE'].sort_values(by = 'Overall', axis = 1, ascending = False)

,case2_lambda_1_mean,case1_lambda_1_mean,case1_lambda_1_median,case2_lambda_1_median,"case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean",bottomup,"case2_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 0.09]_median",...,"case2_lambda_[0.01, 0.09]_median","case2_lambda_[0.01, 5]_median","case2_lambda_[0.01, 5]_mean","case2_lambda_[0.1, 0.9]_mean",wls,"case2_lambda_[0.01, 0.09]_mean",mintshrink,ols,mintsample,erm
Total Employees,64.372412,64.374173,64.011286,63.994620,62.302712,62.306116,61.946867,62.396134,62.397307,64.397293,...,40.992024,64.720746,66.216188,57.712639,66.756058,21.513257,54.053489,23.221948,67.246988,-15.766687
Main Occupation,17.547851,17.547851,17.251575,17.238903,17.447684,17.421737,17.491130,17.232266,17.230455,15.544520,...,26.141657,12.638483,12.249885,13.791002,8.599841,24.428568,6.994762,1.699814,-19.028022,-173.911443
Employment Status,6.076752,6.072756,6.034127,6.030131,6.136694,6.115381,6.179319,5.944880,5.946212,4.783344,...,5.836985,0.976383,0.007992,1.302732,0.682003,7.303558,-0.166505,-2.929149,-11.338297,-189.265115
Gender,-0.201480,-0.195183,-0.097592,-0.110184,-0.022037,-0.040926,0.000000,-0.132221,-0.138517,-1.023139,...,-2.099795,-3.078860,-4.152369,-2.556273,-6.831418,0.166850,-8.695105,-5.383283,-6.242720,-149.013065
Overall,19.385585,19.385585,19.237752,19.227378,19.012112,18.995254,18.971912,18.887621,18.885027,18.243121,...,16.939855,15.912804,15.573048,15.112690,14.141401,13.573411,10.531162,2.969629,2.754364,-141.692818


### Across 5 Samples

ARIMA

In [74]:
sample_precentages = get_results_across_samples(data_labour, model_arima, LEVELS_labour, FC_TYPE, 5, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,ols,mintshrink,wls,case2_lambda_1_mean,case2_lambda_1_median,"case1_lambda_[1, 1.5]_mean","case2_lambda_[1, 1.5]_mean",case1_lambda_1_mean,case1_lambda_1_median,"case2_lambda_[0.01, 5]_mean",...,bottomup,"case2_lambda_[0.1, 0.9]_median","case1_lambda_[0.01, 5]_median","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_mean","case1_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 5]_mean","case2_lambda_[0.01, 0.09]_median",mintsample,erm
Total Employees,-16.736904,-102.266681,-133.427911,-210.206222,-218.927117,-199.769422,-212.627409,-247.353540,-253.789786,-243.543211,...,-315.958814,-325.577252,-327.752375,-330.710178,-320.155046,-346.798827,-342.452129,-327.427373,-413.305079,-1955.577129
Main Occupation,6.852208,-0.744838,-2.217940,-11.149778,-11.814460,-18.872124,-16.010438,-15.846592,-16.366648,-21.601815,...,-21.705324,-21.668805,-26.933388,-27.452457,-32.064452,-29.609559,-30.225274,-32.878469,-64.347205,-496.492064
Employment Status,7.109284,5.587472,4.869467,-4.139635,-4.592360,-6.529533,-5.068258,-4.350228,-4.758324,-6.821333,...,-6.902316,-7.008742,-8.452244,-10.452258,-14.107975,-11.054208,-11.196997,-14.035405,-63.401608,-564.143128
Gender,7.028137,6.248875,6.194292,1.807274,1.373207,-0.423519,0.795288,1.642096,1.388655,0.217239,...,0.000000,0.059300,-1.515214,-1.866401,-4.683118,-2.182386,-2.411493,-4.692051,-70.618148,-547.769374
Overall,4.625514,-6.470093,-9.706742,-21.566323,-23.138652,-24.101539,-26.342030,-28.039382,-28.616175,-28.663454,...,-38.079265,-38.590283,-41.760660,-42.490604,-44.586599,-45.589522,-45.816459,-46.652221,-105.955035,-619.496259


ETS

In [75]:
sample_precentages = get_results_across_samples(data_labour, model_ets, LEVELS_labour, FC_TYPE, 5, True)
sample_precentages.sort_values(by = 'Overall', axis = 1, ascending = False)

,case2_lambda_1_median,case2_lambda_1_mean,"case1_lambda_[0.1, 0.9]_median","case1_lambda_[0.1, 0.9]_mean",ols,"case1_lambda_[1, 1.5]_median","case1_lambda_[1, 1.5]_mean","case1_lambda_[0.01, 5]_median","case2_lambda_[0.1, 0.9]_median",case1_lambda_1_median,...,"case2_lambda_[0.01, 0.09]_median","case1_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 0.09]_mean","case2_lambda_[0.01, 5]_mean","case2_lambda_[1, 1.5]_median","case2_lambda_[1, 1.5]_mean",wls,bottomup,mintsample,erm
Total Employees,-8.986735,-15.615242,-13.224410,-18.834599,-6.573752,-29.517325,-29.259948,-35.604912,-28.660703,-38.903550,...,-35.957065,-40.081324,-41.038014,-52.213650,-46.333455,-45.041686,-41.494882,-56.004525,-76.016173,-890.153458
Main Occupation,11.995196,8.682858,8.595632,8.765707,10.793078,9.833419,9.141019,9.309294,7.431128,9.143494,...,7.121023,7.573981,7.909109,8.607292,10.280405,9.927409,9.308589,6.293989,-122.806951,-1078.469108
Employment Status,7.528445,5.421963,4.178075,4.349830,4.424146,4.957343,4.365076,5.532777,4.008181,4.554856,...,3.354879,3.924545,4.176541,4.758690,4.835096,4.375163,4.029185,3.109709,-131.628524,-1021.454876
Gender,2.901541,1.600443,0.504308,0.786013,0.677271,0.726411,0.524506,1.584128,0.505070,1.142852,...,-0.496385,0.556699,0.242195,0.743173,1.093648,0.793785,0.483063,0.000000,-125.570091,-982.485282
Overall,8.716700,5.612534,5.480109,4.829140,4.045769,2.818042,2.352986,2.346206,1.592957,0.934885,...,-0.604638,-0.628039,-0.652892,-0.938944,-1.795720,-1.851191,-2.025027,-4.613162,-111.257047,-917.270542
